# Real-Time Face Recognition using Face Recognition Module

### Objective:
To understand how to implement real-time face recognition using the `face_recognition` library in combination with OpenCV for detecting and recognizing faces.

### Prerequisites:
- Installed OpenCV library (opencv-contrib-python).

## Step 1: Import the Necessary Libraries
First, we need to import the OpenCV and face_recognition libraries which provide the necessary tools for face detection and recognition.

In [ ]:
import os
import cv2
import numpy as np
import face_recognition

## Step 2: Load Known Faces and Their Encodings
We prepare our dataset of known faces. For each known person, we load their image, find the face encodings, and store them along with the person's name.

In [ ]:
# Path to the directory containing images of known faces
known_faces_dir = 'known_faces'

# Lists to hold encodings and names
known_face_encodings = []
known_face_names = []

# Load each known face and its encoding
for filename in os.listdir(known_faces_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Load the image
        image_path = os.path.join(known_faces_dir, filename)
        image = face_recognition.load_image_file(image_path)
        
        # Get the face encoding
        face_encoding = face_recognition.face_encodings(image)[0]
        
        # Store the encoding and the name
        known_face_encodings.append(face_encoding)
        known_face_names.append(os.path.splitext(filename)[0])

## Step 3: Initialize the Webcam
We open a connection to the webcam using OpenCV's VideoCapture method. The parameter 0 indicates that we want to use the default webcam.

In [ ]:
# Open a connection to the webcam
cap = cv2.VideoCapture(0)

## Step 4: Capture Frames in a Loop

- We continuously capture frames from the webcam in a loop until the user decides to exit.
- The face_recognition library requires frames in RGB format, so we convert the captured frame from BGR to RGB.
- Using the face_recognition library, we detect faces and get their encodings in the captured frame.
- For each detected face, we compare its encoding with the known face encodings to find matches.
- For each detected face, we draw a custom bounding box with corner lines to highlight the detected face. We also display the name of the recognized person.
- We display the frame with the detected faces and the custom bounding boxes in a window named 'Face Detection'.
- We break the loop and stop the webcam feed when the user presses the `q` key on the keyboard.

In [ ]:
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # Convert the frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Detect faces
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
    
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # Compare face encodings
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

        # Find the known face with the smallest distance to the new face
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            label_text = known_face_names[best_match_index]
            color = (0, 255, 0)  # Green for recognized faces
        else:
            label_text = "Unknown"
            color = (0, 0, 255)  # Red for unknown faces
        
        # Draw capture icon style bounding box around detected faces
        line_length = 30  # Length of the corner lines
        thickness = 2  # Thickness of the lines
        
        # Top-left corner
        cv2.line(frame, (left, top), (left + line_length, top), color, thickness)
        cv2.line(frame, (left, top), (left, top + line_length), color, thickness)
        
        # Top-right corner
        cv2.line(frame, (right, top), (right - line_length, top), color, thickness)
        cv2.line(frame, (right, top), (right, top + line_length), color, thickness)
        
        # Bottom-left corner
        cv2.line(frame, (left, bottom), (left + line_length, bottom), color, thickness)
        cv2.line(frame, (left, bottom), (left, bottom - line_length), color, thickness)
        
        # Bottom-right corner
        cv2.line(frame, (right, bottom), (right - line_length, bottom), color, thickness)
        cv2.line(frame, (right, bottom), (right, bottom - line_length), color, thickness)
        
        # Display the name of the detected face
        cv2.putText(frame, label_text, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, thickness)

        # Display the confidence value below the name
        distance_text = f"Match Distance: {face_distances[best_match_index]:.2f}"
        cv2.putText(frame, distance_text, (left, bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, thickness)
    
    # Display the resulting frame
    cv2.imshow('Face Recognition', frame)
    
    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

# When everything done, release the capture
cv2.destroyAllWindows()
cv2.waitKey(1)